# 文字列・時間型操作

ここでは、`pandas` を使った文字列データや時間データの取り扱いについて学びます。

In [1]:
import pandas as pd
from pandas import DataFrame, Series

## 文字列操作

時には、欲しい数値が文字列の一部に入っている場合があります。
以下は、サービスの評価を 1 ~ 5 段階で評価してもらった顧客満足度データを表しています。
サービス評価のデータは、尺度の数値とカテゴリー名がくっついたものになっています。
データとしては何を示しているか分かりやすいですが、満足度の平均値を計算するには数値を取り出す必要があります。

In [2]:
data = DataFrame({
    '顧客ID': [101, 102, 103, 104, 105],
    'サービス評価': ['1.とても悪かった', '2.悪かった', '3.普通', '4.良かった', '5.とても良かった'],
}) # アンケートデータの例（尺度の数値と尺度のカテゴリー名がくっついている）
data

,顧客ID,サービス評価
0,101,1.とても悪かった
1,102,2.悪かった
2,103,3.普通
3,104,4.良かった
4,105,5.とても良かった


`Series` の `str` アクセッサーを使うと、文字列に対する高度な操作が可能です。
`str` アクセッサーは、`Series` オブジェクトに対して `.str` をつけることで、使用可能になります。
`data` が `Series` オブジェクトとすると、`data.str.<method>` で文字列操作のメソッドが使えます。

文字列操作の機能を使う前に、まずはデータのパターンを読み解きます。
サービス評価の値は、`数値.カテゴリー名` というパターンで表記されています。
したがって、いくつかの方策が考えられるはずです。

1. ドット `.` で文字列をリストに分割し、`0` 番目の要素を取り出す
2. 1 桁の数値しかないので、最初の文字だけを切り出す
3. 文字列の先頭から、数値の部分だけを取り出す
4. 数値以外の文字を空文字 `""` にする

ここでは、最初の方法でやってみます。
文字列のメソッドと同じように、`pandas` でも [`split`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html) メソッドでデータの文字列を分割できます。
デフォルトだと、文字列を区切り文字で分割したリストの `Series` が返ってきます。

In [3]:
data["サービス評価"].str.split(".")  # . で文字列を分割

0    [1, とても悪かった]
1       [2, 悪かった]
2         [3, 普通]
3       [4, 良かった]
4    [5, とても良かった]
Name: サービス評価, dtype: object

`str` アクセッサーを使って、`i` 番目の要素を取り出すことができます。
最初の要素を取り出すには、`str[0]` です。
次のように組み合わせることで、求めていた数値が取り出せました。

In [4]:
data["サービス評価"].str.split(".").str[0]

0    1
1    2
2    3
3    4
4    5
Name: サービス評価, dtype: object

ちなみに、他の方法でやる場合は以下の方法で可能です。

2. `str[0]` で最初の文字をとる
3. `extract` メソッドでパターンにマッチする要素 (数値) を取り出す
4. `replace` メソッドでパターンにマッチする要素 (数値以外) を空文字 `""` にする

また、ここでは深く触れませんが、パターンを示すには正規表現 (regular expression) というものを用いることで、より広い範囲のパターンを示すことができます。
例えば、正規表現では任意の数値を `\d` で示せます。
`extract(r"(\d))` とすると、先頭に数値がある場合に、それを取り出します。

## パターンマッチング

パターンマッチングとは、特定の数値・文字列のパターンを検出するものです。
また、`和歌山県和歌山市` という文字列から、`県` の前の文字列 (`和歌山`) を抽出することもできます。

例えば次のような商品の `Series` があるとします。

In [6]:
products = Series([
  "ポテトサラダ",
  "じゃがいも",
  "カルビー ポテトチップス うすしお味",
  "湖池屋 ポテトチップス のり塩味", 
  "海藻サラダ",
])
products

0                ポテトサラダ
1                 じゃがいも
2    カルビー ポテトチップス うすしお味
3      湖池屋 ポテトチップス のり塩味
4                 海藻サラダ
dtype: object

パターンを検出するメソッドは、`fullmatch`, `match`, `contains` などがあります。
これらは、指定したパターンに一致している文字列の時は `True`、そうでないときは `False` を返します。
メソッドによって、若干の差異があります。

- fullmatch: 指定したパターンと文字列が完全に一致した場合のみ `True` を返す
- match: 指定したパターンと先頭の文字列が一致した場合に `True` を返す
- contains: 指定したパターンを文字列が含む場合に `True` を返す

先ほどのデータを使って実際に試してみます。
まずは、ポテトという文字列とマッチさせます。

In [7]:
products.str.fullmatch("ポテト")

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [8]:
products.str.match("ポテト")

0     True
1    False
2    False
3    False
4    False
dtype: bool

In [9]:
products.str.contains("ポテト")

0     True
1    False
2     True
3     True
4    False
dtype: bool

`fullmatch` では、ポテトという商品名はないため、全てが `False` になっています。

`match` ではポテトサラダは先頭に「ポテト」の文字列があるため `True` になっていますが、ポテトチップスはその前にブランド名があるため `False` が返っています。

`contains` はポテトチップスも `True` を返します。

クイズ

- `fullmatch` を使って「ポテトサラダ」という文字列パターンとマッチしてください
- ポテトチップスだけを検出してみてください
- じゃがいもに関連する商品全てを検出してください (つまり、海藻サラダ以外)

## 日付・時間型データの操作

ユーザーの登録日時や購入時間などでは、時間のデータ型 `datetime64` を用いるのが便利です。

In [10]:
sales_data = DataFrame({
    '顧客ID': [1, 2, 1, 3, 2],
    '購入日時': ['2023-01-01 10:00', '2023-01-05 15:30', 
                 '2023-02-01 11:00', '2023-02-05 18:00', 
                 '2023-03-01 09:30'],
    '購入金額': [1000, 1500, 2000, 2500, 3000]
})
sales_data["購入日時"]  # object 型

0    2023-01-01 10:00
1    2023-01-05 15:30
2    2023-02-01 11:00
3    2023-02-05 18:00
4    2023-03-01 09:30
Name: 購入日時, dtype: object

`datetime64` 型の `Series` の作成は、`pandas` の `to_datetime` 関数を使うことで可能です。

In [11]:
sales_data["購入日時"] = pd.to_datetime(sales_data["購入日時"])  # datetime64 型に変換
sales_data["購入日時"]

0   2023-01-01 10:00:00
1   2023-01-05 15:30:00
2   2023-02-01 11:00:00
3   2023-02-05 18:00:00
4   2023-03-01 09:30:00
Name: 購入日時, dtype: datetime64[ns]

`dt` アクセッサーを使うことで、年、月、日などの情報を取り出すことができます。

In [12]:
sales_data["購入日時"].dt.year  # 年を取得

0    2023
1    2023
2    2023
3    2023
4    2023
Name: 購入日時, dtype: int32

In [13]:
sales_data["購入日時"].dt.day_of_week  # 曜日の数字 

0    6
1    3
2    2
3    6
4    2
Name: 購入日時, dtype: int32

曜日の数字は、月曜 (0) から始まり、日曜 (6) で終わります。

https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.day_of_week.html

また、[`resample`](https://pandas.pydata.org/docs/reference/api/pandas.Series.resample.html) メソッドを使うと、指定した時間の単位でデータを集計してくれます。
`groupby` の時間版だと思ってください。

例えば、日次平均、年次平均を計算することが可能です。
`resample` メソッドでは、集計する期間を指定します。
`"M"` は月単位を表し、`"Y"` は年単位を表します。
index が時間ではない場合、`on` キーワードで集計に用いる時間のカラム名を指定します。

`resample` で集計した後に、集計メソッドを適用します。

In [14]:
sales_data

,顧客ID,購入日時,購入金額
0,1,2023-01-01 10:00:00,1000
1,2,2023-01-05 15:30:00,1500
2,1,2023-02-01 11:00:00,2000
3,3,2023-02-05 18:00:00,2500
4,2,2023-03-01 09:30:00,3000


In [18]:
sales_data.resample("ME", on = "購入日時").mean() # 月次平均

,顧客ID,購入金額
購入日時,,
2023-01-31,1.5,1250.0
2023-02-28,2.0,2250.0
2023-03-31,2.0,3000.0


In [19]:
sales_data.resample("YE", on="購入日時").mean()  # 年次平均

,顧客ID,購入金額
購入日時,,
2023-12-31,1.8,2000.0


さらに、`"5Min"` とすれば 5 分毎に集計するなど、柔軟な指定が可能です。
詳しくは、`pandas` のドキュメントを読んでください。

- https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html